In [1]:
#import some stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import glob
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split #use this?
from scipy import stats
from sklearn.metrics import log_loss, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV

In [2]:
urlScout = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_scouting.csv'
urlWomen = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_womens.csv'
dataScout = pd.read_csv(urlScout)
dataWomen = pd.read_csv(urlWomen)

In [3]:
dataScoutxG = pd.read_csv('dataScoutwithxG.csv')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
dataScoutxG.head(1)

,Unnamed: 0,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2,xG
0,0,2019-09-20,Erie Otters,Sudbury Wolves,1,20:00,5,5,0,0,Sudbury Wolves,Blake Murray,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN,0.013987


In [6]:
def getWPAready(df):
    df = df.copy()
    df['TimeLeft'] = df.apply(lambda row: (20*(3 - float(row['Period'])))*(float(row['Clock'].split(':')[0]) * 60 + float(row['Clock'].split(':')[1])),axis=1)
    df['HomeExtraGoals'] = df['Home Team Goals'] - df['Away Team Goals']
    df['HomexG'] = df.apply(lambda row: row['xG'] if (row['Team'] == row['Home Team']) else (-1*row['xG']),axis=1)
    df['NextEventGameChange'] = (df['game_date'] != df['game_date'].shift(-1))
    HomeWinDic = {}
    finalEvents = df.loc[df['NextEventGameChange']]
    for i in range(len(finalEvents)):
        row = finalEvents.iloc[i]
        if (row['Home Team Goals'] < row['Away Team Goals']):
            HomeWinDic[row['game_date']] = 0
        elif (row['Home Team Goals'] == row['Away Team Goals']):
            HomeWinDic[row['game_date']] = 1
        else:
            HomeWinDic[row['game_date']] = 2
    df['HomeWin'] = df['game_date'].map(HomeWinDic)
    df = df[['TimeLeft','HomeExtraGoals','HomexG','HomeWin']]
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X,y

In [7]:
X,y = getWPAready(dataScoutxG)

In [8]:
print("Overall Mean:", y.mean())
#get train/test split ready

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.25,random_state=8) #can add shuffle=False to not shuffle

#train the xgboost model
dtrain = xgb.DMatrix(Xtrain,label=ytrain)

# XGBoost model
nrounds = 560 #use gridSearchCV
params = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'booster' : 'gbtree', #gblinear/gbtree?
    'eval_metric' : ['logloss'],
    'eta' : 0.1,
    'gamma' : 3,
    'subsample' : 0.8,
    'colsample_bytree' : 0.8,
    'max_depth': 6,
    'min_child_weight' : 6,
    'base_score' : ytrain.mean()
}

wp_model = xgb.train(params, dtrain, num_boost_round=nrounds, verbose_eval=2)
# save for debug
wp_model.save_model('wpModel.json')

Overall Mean: 0.8259987083679307


In [10]:
%%time
#test the model

dtest = xgb.DMatrix(Xtest,label=ytest)
wp_predictions = wp_model.predict(dtest) #.tolist()
#print('time to test model is ',time.time() - start_time)

Wall time: 560 ms


In [11]:
preds0,preds1,preds2 = wp_predictions[:,0],wp_predictions[:,1],wp_predictions[:,2]
XtestCopy = Xtest.copy()
XtestCopy['xHomeLose'] = preds0
XtestCopy['xHomeTie'] = preds1
XtestCopy['xHomeWin'] = preds2
#Home team gets 2 points for win, 1 for tie, 0 for loss
XtestCopy['xHomePointsWon'] = XtestCopy.apply(lambda row: row['xHomeTie'] + 2*row['xHomeWin'],axis=1)
print('xHomePoints Mean vs. actual mean')
print(XtestCopy['xHomePointsWon'].mean())
print(ytest.mean())

xHomePoints Mean vs. actual mean
0.8268952387344112
0.8231324793083452


In [12]:
allData = xgb.DMatrix(X)
allDatawp_predictions = wp_model.predict(allData) #.tolist()

preds0,preds1,preds2 = allDatawp_predictions[:,0],allDatawp_predictions[:,1],allDatawp_predictions[:,2]
XCopy = X.copy()
XCopy['xHomeLose'] = preds0
XCopy['xHomeTie'] = preds1
XCopy['xHomeWin'] = preds2
#Home team gets 2 points for win, 1 for tie, 0 for loss
XCopy['xHomePointsWon'] = XCopy.apply(lambda row: row['xHomeTie'] + 2*row['xHomeWin'],axis=1)
print('xHomePoints Mean vs. actual mean')
print(XCopy['xHomePointsWon'].mean())
print(y.mean())

xHomePoints Mean vs. actual mean
0.8269808318906651
0.8259987083679307


In [13]:
df = dataScoutxG.copy()
XCopy = XCopy[['xHomePointsWon']]
df = df.merge(XCopy,how='left',left_index=True,right_index=True)

In [20]:
def getWPA(df):
    df = df.copy()
    df['WPA'] = df['xHomePointsWon'] - df['xHomePointsWon'].shift(-1)
    #now change it for away team 
    df['WPA'] = df.apply(lambda row: row['WPA'] if (row['Team'] == row['Home Team']) else -1*row['WPA'],axis=1)
    #change start of game to 0 WPA added
    df['LastOfGame'] = (df['game_date'] != df['game_date'].shift(-1))
    df['WPA'] = df.apply(lambda row: 0 if (row['LastOfGame']) else row['WPA'],axis=1)
    return df

In [16]:
dataScout.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2019-09-20,Erie Otters,Sudbury Wolves,1,20:00,5,5,0,0,Sudbury Wolves,Blake Murray,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN
1,2019-09-20,Erie Otters,Sudbury Wolves,1,19:58,5,5,0,0,Sudbury Wolves,Emmett Serensits,Puck Recovery,67,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-09-20,Erie Otters,Sudbury Wolves,1,19:53,5,5,0,0,Sudbury Wolves,Emmett Serensits,Incomplete Play,45,49,Direct,NaN,NaN,NaN,Blake Murray,74.0,35.0
3,2019-09-20,Erie Otters,Sudbury Wolves,1,19:52,5,5,0,0,Erie Otters,Hayden Fowler,Puck Recovery,89,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-09-20,Erie Otters,Sudbury Wolves,1,19:51,5,5,0,0,Erie Otters,Hayden Fowler,Incomplete Play,82,49,Direct,NaN,NaN,NaN,Connor Lockhart,94.0,46.0


In [21]:
wpadf = getWPA(df)

In [22]:
wpadf.head(5)

,Unnamed: 0,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2,xG,xHomePointsWon,WPA,LastOfGame
0,0,2019-09-20,Erie Otters,Sudbury Wolves,1,20:00,5,5,0,0,Sudbury Wolves,Blake Murray,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN,0.013987,0.628318,0.000220,False
1,1,2019-09-20,Erie Otters,Sudbury Wolves,1,19:58,5,5,0,0,Sudbury Wolves,Emmett Serensits,Puck Recovery,67,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009805,0.628538,0.098408,False
2,2,2019-09-20,Erie Otters,Sudbury Wolves,1,19:53,5,5,0,0,Sudbury Wolves,Emmett Serensits,Incomplete Play,45,49,Direct,NaN,NaN,NaN,Blake Murray,74.0,35.0,0.009502,0.726945,0.363200,False
3,3,2019-09-20,Erie Otters,Sudbury Wolves,1,19:52,5,5,0,0,Erie Otters,Hayden Fowler,Puck Recovery,89,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013689,1.090145,0.157114,False
4,4,2019-09-20,Erie Otters,Sudbury Wolves,1,19:51,5,5,0,0,Erie Otters,Hayden Fowler,Incomplete Play,82,49,Direct,NaN,NaN,NaN,Connor Lockhart,94.0,46.0,0.013602,0.933031,0.050560,False


In [23]:
wpadf = wpadf.loc[wpadf['Team'] == 'Erie Otters']
wpadf = wpadf[['Player','WPA']]
wpadf.groupby(by=['Player']).sum()

,WPA
Player,
Aidan Campbell,-0.321206
Alex Gritz,1.138951
Austen Swankler,19.673155
Brendan Hoffmann,8.600888
Brendan Kischnick,3.029400
Brendan Sellan,17.093018
Brett Bressette,1.662680
Cameron Morton,5.784486
Chad Yetman,8.501839
